In [2]:
%load_ext dotenv
%dotenv

In [78]:
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

In [30]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [32]:
chat = ChatOpenAI(model_name = 'gpt-4o-mini', 
                  seed = 365,
                  temperature = 0,
                  max_tokens = 100)

In [34]:
chain = prompt_template | chat | StrOutputParser()

In [36]:
chain.invoke({'message_log':'''The AI provides an interesting fact about octopuses, stating that they have three hearts. 
Two hearts pump blood to the gills, while the third heart pumps it to the rest of the body.''', 
              'question':"Can you elaborate a bit more on this fact?"})

'Absolutely! Octopuses are fascinating creatures, and their unique circulatory system is one of the reasons why. The three hearts serve distinct purposes: \n\n1. **Branchial Hearts**: The two hearts that pump blood to the gills are called branchial hearts. They are responsible for ensuring that the blood is oxygenated as it passes through the gills, where gas exchange occurs. This is crucial because octopuses live in aquatic environments and need a constant supply of oxygen.\n\n2. **'

In [37]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(), memory_key = 'message_log')

In [38]:
chat_memory.load_memory_variables({})

{'message_log': ''}

In [76]:
RunnablePassthrough.assign(message_log = chat_memory.load_memory_variables).invoke(
    {'question':"Can you give me an interesting fact I probably didn't know about?"})

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_log': {'message_log': ''}}

In [74]:
RunnablePassthrough.assign(first_letter=lambda x: list(x['input'])[0],
                           second_letter=lambda x: list(x['input'])[1]
                          ).invoke({'input':'hi'})

{'input': 'hi', 'first_letter': 'h', 'second_letter': 'i'}

In [86]:
RunnableLambda(itemgetter('message_log')).invoke({'message_log':''})

''

In [94]:
RunnablePassthrough.assign(message_log = 
                           RunnableLambda(chat_memory.load_memory_variables) | 
                           RunnableLambda(itemgetter('message_log')) ).invoke(
    {'question':"Can you give me an interesting fact I probably didn't know about?"})

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_log': ''}